In [ ]:
# !earthengine authenticate

# NOTE: instead of running this in the notebook use the binder terminal
# 1. go to File > new > terminal
# 2. enter `earthengine authenticate` & follow prompts

In [1]:
cd /home/jovyan

/home/jovyan


In [ ]:
# check to ensure the file we need is there
!ls -lh '/home/jovyan/.config/earthengine/credentials'

#### Import EE

In [1]:
import ee
import geemap
ee.Initialize()

In [2]:
## Verify the ee module is loaded.
print(ee.__version__)

0.1.286


#### Import asset collections

Ocean Color and SST4 daily product (2002-Present) from MODIS at 1 km per pixel (FLORIDA).

OC Bands:\
b1 = Rrs 412, b2 = Rrs 443, b3 = Rrs 488, b4 = Rrs 531, b5 = Rrs 547, b6 = Rrs 667, b7 = Rrs 678, b8 = chlor a (mg/m3), b9 = nflh

SST4 Bands:\
b1 = SST (°C)

In [3]:
## Florida MODIS L3 data (uploaded from imars server)
oc = ee.ImageCollection('projects/imars-3d-wetlands/MODIS_L3/FL_OC') ## Ocean Color
sst = ee.ImageCollection('projects/imars-3d-wetlands/MODIS_L3/FL_SST4') ## SST

#### Monthly means

Define some variables first

In [7]:
band = 'b1' #Type same band in Line 149
renameBand = ['SST']
collection = sst
start_date = '2003-01-01'
end_date = '2020-12-31'

# Establish lists with months and years of selected time range above.
months = ee.List.sequence(1, 12);
years = ee.List.sequence(2003, 2020);

# Establish lists with months and years of selected time range above.
months = ee.List.sequence(1, 12);
years = ee.List.sequence(2003, 2020);

## lat/lon limits
region = polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

## Visualization parameters
palette = ['030d81','0519ff','05e8ff','11ff01','fbff01','ff9901','ff0000','ad0000']
viz = {'min':25,'max':31,'palette':palette}

Filter collection and get monthly means:

In [8]:
## Filter collection
filtered = collection.filterDate(ee.Date(start_date), ee.Date(end_date)).select(band)

## Get monthly menas in the defined time range
byMonthYear = ee.ImageCollection.fromImages(
  years.map(lambda y: 
            months.map(lambda m:
                              filtered\
                              .filter(ee.Filter.calendarRange(y, y, 'year'))\
                              .filter(ee.Filter.calendarRange(m, m, 'month'))\
                              .reduce(ee.Reducer.mean())\
                              .set('month', m).set('year', y))).flatten())

print('Monthly Means:',byMonthYear.size().getInfo())

Monthly Means: 216


In [12]:
# initialize our map
Map = geemap.Map()
Map.centerObject(region, 6)
Map.addLayer(byMonthYear.first(), viz, "Florida")

Map.addLayerControl()
Map

#Map.addLayer(byMonthYear.first(),vis_chlor,'image')
#

Map(center=[-17.01610988343402, -27.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [9]:
## Function to Get mean and stdv values
def getStats(img):
    ## Select and rename bands.
    #image = ee.Image(img).select(band+'_mean').rename(renameBand)
    image = ee.Image(img).rename(renameBand)
    ## Create and combine reducers of mean and std.
    reducers = ee.Reducer.mean().combine(
        reducer2= ee.Reducer.stdDev(),
        sharedInputs= True)
    ## Apply reducers to the image.
    stats = image.reduceRegion(
        reducer= reducers,
        geometry= region,
        #bestEffort= True,
        scale= 1000,
        maxPixels= 1e9)
    
    ## Create feature variable to allocate data.
    feature = ee.Feature(None)
    ## Set properties of interest.
    properties = ['system:index','month','year',renameBand+'_mean',renameBand+'_stdDev']
    ## Apply the stats on every image and collect the output.
    imageStats = image.setMulti(stats)
    ## return feature with the selected properties.
    return ee.Feature(feature).copyProperties(imageStats, properties)

In [10]:
stats = byMonthYear.map(getStats)

TypeError: can only concatenate list (not "str") to list